In [1]:
import sys, os, json
sys.path.append('GPU_Based_2')
import cuda_v7 as cv7
sys.path.append('CPU_Based')
import cuda_to_go as ctg
import numpy as np
from scipy import special
import time, random, math
from numba import cuda

In [2]:
# Define useful constants
amu = 1.67e-27
eps0 = 8.854e-12
qe = 1.6e-19 # SI units 

# Define physical params
m = 40. *amu
q = 1. *qe
wr = 2*np.pi*3e6 # SI units

# Define sim params
Dr = 30001.5e-9
Dz = 90001.5e-9 # physical width in m of the sim

# wz can be varied if desired
wz = 2*np.pi*1e6

aH2 = 8e-31 # dipole moment of H2 in SI units
mH2 = 2.0*amu # mass of H2 in kg

dtSmall = 1e-12
dtCollision = 1e-16
dtLarge = 1e-10 # length of a time step in s

sigmaV = 100e-6 # fall-off of potential outside trapping region
dv = 20.0 # bin size for particle speed in determining if collision occurs
vmax = 5000 # maximum particle speed we allow
l = 1
vbumpr = 0.00e0
vbumpz = -0.0e0 # starting velocity in r and z of the lth ion in the chain
offsetz = 0.0e-7
offsetr = 0.0e-8 # starting distance from eq. in r and z of the lth ion in the chain

velocity = 2000
angle_choice = np.pi/4
offsets = .5e-9
max_hypotenuse = 1.5e-5
grid_size = 2001
Ni = 2
shot_sizes = [1]
ion_collided = 0

In [3]:
"""
    CPU Version
"""

print(f"Starting CPU Version!")

Nr = Nz = grid_size
Nrmid = Nzmid = (Nr-1)/2
dr = Dr/float(Nr)
dz = Dz/float(Nz)

# Recalculate RF and DC fields for each grid size
RF = ctg.makeRF0(m, q, wr, Nr, Nz, Nrmid, dr)
DC = ctg.makeDC(m, q, wz, Nz, Nr, Nzmid, dz)


start_time = time.perf_counter()

for shots in shot_sizes:
    print(f"Starting simulation with grid size {grid_size}, ion count {Ni}, and shots {shots}")
    #print(f"Simulation Size = {Dr} m in r, {Dz} m in z")


    for i in range(shots):
        vf = ctg.makeVf(Ni, 1.0*q, m, l, wz, offsetr, offsetz, vbumpr, vbumpz)

        Nt = 10000

        r = -np.cos(angle_choice)*max_hypotenuse
        z = vf[ion_collided,1] + np.sin(angle_choice)*max_hypotenuse + offsets
        vz = -1*velocity*np.sin(angle_choice)
        vr = np.abs(velocity*np.cos(angle_choice))

        reorder = ctg.mcCollision(vf,r,z,vr,vz,q,mH2,aH2,Nt,dtSmall,RF,DC,Nr,Nz,dr,dz,dtLarge,dtCollision)


        finish_time = time.perf_counter()
        timeTaken = finish_time - start_time

print(reorder)


Starting CPU Version!
Starting simulation with grid size 2001, ion count 2, and shots 1


















































































































































































































































































































































































































































































































































































































































































































































































































































































































































In [4]:
# """
#     GPU Version
# """

# print(f"Starting GPU Version!")

# Nr = Nz = grid_size
# Nrmid = Nzmid = (Nr - 1) // 2
# dr = Dr / float(Nr)
# dz = Dz / float(Nz)

# RF = cv7.makeRF0(m, q, wr, Nr, Nz, Nrmid, dr)
# DC = cv7.makeDC(m, q, wz, Nz, Nr, Nzmid, dz)

# RF_device = cuda.to_device(RF)
# DC_device = cuda.to_device(DC)
# nullFields_device = cuda.to_device(np.zeros((Nr, Nz), dtype=np.float64))

# ion_counts = [2]
# for Ni in ion_counts:
#     for shots in shot_sizes:
#         print(f"Starting simulation with grid size {grid_size}, ion count {Ni}, and shots {shots}")

#         start_time = time.perf_counter()

#         vf_all = []
#         rc_all = []
#         zc_all = []
#         vrc_all = []
#         vzc_all = []
#         Nt_all = []
#         ion_collided_all = []  # To store ion collided indices

#         for i in range(shots):
#             vf = cv7.makeVf(Ni, q, m, l, wz, offsetr, offsetz, vbumpr, vbumpz)

#             Nt = 100

#             r = math.cos(angle_choice) * -max_hypotenuse
#             z = vf[ion_collided, 1] + math.sin(angle_choice) * max_hypotenuse + offsets
#             vz = -1 * velocity * math.sin(angle_choice)
#             vr = math.fabs(velocity * math.cos(angle_choice))

#             vf_all.append(vf)
#             rc_all.append(r)
#             zc_all.append(z)
#             vrc_all.append(vr)
#             vzc_all.append(vz)
#             Nt_all.append(Nt)
#             ion_collided_all.append(ion_collided)  # Store the collided ion index

#         vf_all = np.array(vf_all, dtype=np.float64)
#         rc_all = np.array(rc_all, dtype=np.float64)
#         zc_all = np.array(zc_all, dtype=np.float64)
#         vrc_all = np.array(vrc_all, dtype=np.float64)
#         vzc_all = np.array(vzc_all, dtype=np.float64)
#         Nt_all = np.array(Nt_all, dtype=np.int32)
#         ion_collided_all = np.array(ion_collided_all, dtype=np.int32)  # Convert to numpy array

#         vf_all_device = cuda.to_device(vf_all)
#         rc_all_device = cuda.to_device(rc_all)
#         zc_all_device = cuda.to_device(zc_all)
#         vrc_all_device = cuda.to_device(vrc_all)
#         vzc_all_device = cuda.to_device(vzc_all)
#         Nt_all_device = cuda.to_device(Nt_all)
#         reorder_all_device = cuda.device_array(shots, dtype=np.int32)

#         threads_per_block = 128
#         blocks_per_grid = (shots + (threads_per_block - 1)) // threads_per_block

#         cv7.mcCollision_kernel[blocks_per_grid, threads_per_block](
#             vf_all_device,
#             rc_all_device,
#             zc_all_device,
#             vrc_all_device,
#             vzc_all_device,
#             q,
#             mH2,
#             aH2,
#             Nt_all_device,
#             dtSmall,
#             RF_device,
#             DC_device,
#             Nr,
#             Nz,
#             dr,
#             dz,
#             dtLarge,
#             dtCollision,
#             nullFields_device,  # Pass nullFields_device
#             reorder_all_device
#         )

#         reorder_all = reorder_all_device.copy_to_host().tolist()  # Convert to Python list

# print(reorder_all)

Starting GPU Version!
Starting simulation with grid size 2001, ion count 2, and shots 1


c:\Users\caleb\anaconda3\envs\collisionEnv\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


TypeError: Signature mismatch: 1 argument types given, but function takes 2 arguments